In [ ]:
import modulos as md

#lista=md.filtradodemonedas()

lista = ['BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'ALGOUSDT', 
         'KNCUSDT', 'COMPUSDT', 'DOGEUSDT', 'KAVAUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'CRVUSDT', 'SOLUSDT', 'STORJUSDT', 'UNIUSDT', 
         'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'AXSUSDT', 'SANDUSDT', 'XEMUSDT', 'MANAUSDT', 'LINAUSDT', 
         'MTLUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', '1000XECUSDT', 'GALAUSDT', 'FLOWUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 
         'APTUSDT', 'FETUSDT', 'RNDRUSDT', 'AGIXUSDT', 'PHBUSDT', 'CFXUSDT', 'STXUSDT', 'LQTYUSDT', 'ARBUSDT', 'SUIUSDT', '1000PEPEUSDT', 'XVGUSDT']


In [1]:
###################################### ESTRATEGIA BB
###################################################################################################################
import modulos as md
lista=['XRPUSDT',	
'EOSUSDT',	
'XTZUSDT',	
'OMGUSDT',	
'CRVUSDT',	
'SUSHIUSDT',
'ENJUSDT',	
'CTKUSDT',	
'1INCHUSDT',
'MANAUSDT',	
'IMXUSDT',	
'MINAUSDT',	
'PERPUSDT',	
'IDUSDT',	
'HFTUSDT',	
'COMBOUSDT',
'MAVUSDT']

#lista=md.lista_de_monedas ()

for symbol in lista:    
    try:
        #data = md.sigo_variacion_bitcoin(symbol)
        data = md.estrategia_bb(symbol)
        resultado = md.backtesting(data, plot_flag = False)
        #if (resultado['Profit Factor'] > 1 
        #    and resultado['Return [%]']/resultado['# Trades'] >=0.3 
        #    and resultado['Win Rate [%]']>50):
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


XRPUSDT - Return [%]: 5.08% - # Trades: 5 - Profit Factor: 11.71
EOSUSDT - Return [%]: -5.99% - # Trades: 9 - Profit Factor: 0.1
XTZUSDT - Return [%]: 0.33% - # Trades: 8 - Profit Factor: 1.2
OMGUSDT - Return [%]: -2.42% - # Trades: 5 - Profit Factor: 0.3
CRVUSDT - Return [%]: 1.57% - # Trades: 8 - Profit Factor: 1.5
SUSHIUSDT - Return [%]: 2.09% - # Trades: 7 - Profit Factor: 1.68
ENJUSDT - Return [%]: -0.47% - # Trades: 7 - Profit Factor: 0.56
CTKUSDT - Return [%]: -2.81% - # Trades: 7 - Profit Factor: 0.28
1INCHUSDT - Return [%]: -0.82% - # Trades: 3 - Profit Factor: 0.0
MANAUSDT - Return [%]: 0.86% - # Trades: 7 - Profit Factor: 1.86
IMXUSDT - Return [%]: -4.97% - # Trades: 5 - Profit Factor: 0.0
MINAUSDT - Return [%]: -2.38% - # Trades: 9 - Profit Factor: 0.38
PERPUSDT - Return [%]: 1.18% - # Trades: 7 - Profit Factor: 2.61
IDUSDT - Return [%]: -1.73% - # Trades: 9 - Profit Factor: 0.17
HFTUSDT - Return [%]: -1.97% - # Trades: 11 - Profit Factor: 0.29
COMBOUSDT - Return [%]: 0.99%